In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale = 1/255, validation_split = 0.1)

train_datagen = ImageDataGenerator(rescale = 1/255, validation_split = 0.1, horizontal_flip = True, vertical_flip = True)

trainGenerator = train_datagen.flow_from_directory(
                                                   r'C:\Users\capnp\OneDrive\Desktop\pneumonia_Xray\chest_xray\chest_xray\train', 
                                                   target_size = (300, 300), 
                                                   batch_size = 32, 
                                                   class_mode = 'categorical',
                                                   subset = 'training')

valGenerator = test_datagen.flow_from_directory(
                                                r'C:\Users\capnp\OneDrive\Desktop\pneumonia_Xray\chest_xray\chest_xray\test', 
                                                target_size = (300, 300), 
                                                batch_size = 32, 
                                                class_mode = 'categorical',
                                                subset = 'validation')

Found 4695 images belonging to 2 classes.
Found 62 images belonging to 2 classes.


In [2]:
classes = (trainGenerator.class_indices)
print(classes)

{'NORMAL': 0, 'PNEUMONIA': 1}


In [3]:
import tensorflow as tf 

cnn = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding = 'same', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(32, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(256, activation='relu'),
   
    tf.keras.layers.Dense(2, activation='softmax')
])

In [4]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 64)        3

In [5]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if(logs.get('val_loss')<0.3):
            print('\n\nReached required accuracy. Stopping the training.')
            self.model.stop_training = True
            
cb = myCallback()

cnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

hist = cnn.fit_generator(trainGenerator, epochs = 5, validation_data = valGenerator, callbacks = [cb])

Epoch 1/5
147/147 [==============================] - 261s 2s/step - loss: 0.3228 - accuracy: 0.8503 - val_loss: 0.4103 - val_accuracy: 0.7258
Epoch 2/5
147/147 [==============================] - 266s 2s/step - loss: 0.1384 - accuracy: 0.9485 - val_loss: 0.6752 - val_accuracy: 0.7258
Epoch 3/5
147/147 [==============================] - 265s 2s/step - loss: 0.1214 - accuracy: 0.9570 - val_loss: 0.5733 - val_accuracy: 0.6613
Epoch 4/5
147/147 [==============================] - 269s 2s/step - loss: 0.0962 - accuracy: 0.9651 - val_loss: 0.4800 - val_accuracy: 0.7419
Epoch 5/5
147/147 [==============================] - 266s 2s/step - loss: 0.0828 - accuracy: 0.9698 - val_loss: 1.0060 - val_accuracy: 0.6613


In [6]:
from keras.preprocessing import image
import numpy as np

path = r'C:\Users\capnp\OneDrive\Desktop\pneumonia_Xray\chest_xray\val\NORMAL\NORMAL2-IM-1442-0001.jpeg'
img = image.load_img(path, target_size=(300, 300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = cnn.predict(images, batch_size=10)
print(classes)

Using TensorFlow backend.


[[1. 0.]]


In [7]:
path = r'C:\Users\capnp\OneDrive\Desktop\pneumonia_Xray\chest_xray\val\PNEUMONIA\person1950_bacteria_4881.jpeg'
img = image.load_img(path, target_size=(300, 300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = cnn.predict(images, batch_size=10)
print(classes)

[[0. 1.]]


# Accuracy of this model is ~96%!